## Process Outlaw data in python before passing to R for optimization

In [7]:
import pandas as pd
import numpy as np

In [8]:
#Read salary data from dk
#6/23
url1 = 'https://www.draftkings.com/lineup/getavailableplayerscsv?contestTypeId=84&draftGroupId=36302'

#6/24
url2 = 'https://www.draftkings.com/lineup/getavailableplayerscsv?contestTypeId=85&draftGroupId=36355'

url3 = 'https://www.draftkings.com/lineup/getavailableplayerscsv?contestTypeId=86&draftGroupId=36414'
df_sal = pd.read_csv(url3)
df_sal.head()

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
0,G,John Catlin (14679194) (LOCKED),John Catlin,14679194,G,10700,The Reno Open,Golf,52.5
1,G,Matt Picanso (14679195) (LOCKED),Matt Picanso,14679195,G,10400,The Reno Open,Golf,0.0
2,G,Mark Anguiano (14679196) (LOCKED),Mark Anguiano,14679196,G,10200,The Reno Open,Golf,0.0
3,G,Alistair Docherty (14679197) (LOCKED),Alistair Docherty,14679197,G,9900,The Reno Open,Golf,0.0
4,G,Norman Xiong (14679198) (LOCKED),Norman Xiong,14679198,G,9700,The Reno Open,Golf,0.0


In [9]:
df_sal[df_sal.Name.str.contains('Smith')]

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
18,G,Zach Smith (14679212) (LOCKED),Zach Smith,14679212,G,8100,The Reno Open,Golf,0.0
40,G,Scott Smith (14679234) (LOCKED),Scott Smith,14679234,G,6800,The Reno Open,Golf,0.0


In [10]:
#read in odds
df_odds = pd.read_csv('Data/Outlaw_odds_20200623.csv')[['Name','Odds']]
df_odds.loc[len(df_odds)]=['Eddie Olson',36]
df_odds.head(12)

,Name,Odds
0,John Catlin,8
1,Rico Hoey,12
2,Matt Picanso,14
3,Mark Anguiano,16
4,Brady Calkins,16
5,Alistair Docherty,20
6,Justin Suh,22
7,Sam Triplett,25
8,MJ Daffue,28
9,Nicolo Galletti,28


In [11]:
df_odds.loc[len(df_odds)] = ['Zach Smith',20]
df_odds[df_odds.Name.str.contains('Smith')]

,Name,Odds
69,Stuart Smith,300
84,Scott Smith,400
143,Zach Smith,20


In [12]:
#replace value in df_odds with closes match in df_dk
#Replace Name in df_odds with closest name in df_sal
from difflib import get_close_matches
for j in np.arange(len(df_odds)):
    curr_name = df_odds['Name'].iloc[j]
    try:
        new_name = get_close_matches(curr_name,df_sal['Name'],cutoff=.8)[0]
        if new_name != curr_name:
            print("{} replaced with {}".format(curr_name,new_name))
    except:
        new_name = 'No Name'
        print('NO MATCH: curr_name = {}, new_name = {}'.format(curr_name,new_name))
    df_odds['Name'].iloc[j] = new_name

df_odds.head()

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


NO MATCH: curr_name = Rico Hoey, new_name = No Name
NO MATCH: curr_name = Justin Suh, new_name = No Name
NO MATCH: curr_name = Sam Triplett, new_name = No Name
NO MATCH: curr_name = MJ Daffue, new_name = No Name
NO MATCH: curr_name = Nicolo Galletti, new_name = No Name
NO MATCH: curr_name = Trevor Sluman, new_name = No Name
NO MATCH: curr_name = Neil Johnson, new_name = No Name
NO MATCH: curr_name = Corey Pereira, new_name = No Name
NO MATCH: curr_name = Gregor Main, new_name = No Name
NO MATCH: curr_name = Armando Favela, new_name = No Name
NO MATCH: curr_name = Kolton Lapa, new_name = No Name
NO MATCH: curr_name = Derek Barron, new_name = No Name
NO MATCH: curr_name = Kyle Westmoreland, new_name = No Name
NO MATCH: curr_name = James Erkenbeck, new_name = No Name
NO MATCH: curr_name = Barry Enright, new_name = No Name
NO MATCH: curr_name = Jhared Hack, new_name = No Name
NO MATCH: curr_name = Jonathan Ruiz, new_name = No Name
NO MATCH: curr_name = Bennett Doyle, new_name = No Name
NO 

,Name,Odds
0,John Catlin,8
1,No Name,12
2,Matt Picanso,14
3,Mark Anguiano,16
4,Brady Calkins,16


In [13]:
#Get df with just stuff we care about
df = df_sal[['Name + ID','Name','Salary']]
df = pd.merge(df,df_odds,on='Name',how='left').dropna()
df.iloc[np.r_[0:5,-3:0]]

,Name + ID,Name,Salary,Odds
0,John Catlin (14679194) (LOCKED),John Catlin,10700,8.0
1,Matt Picanso (14679195) (LOCKED),Matt Picanso,10400,14.0
2,Mark Anguiano (14679196) (LOCKED),Mark Anguiano,10200,16.0
3,Alistair Docherty (14679197) (LOCKED),Alistair Docherty,9900,20.0
4,Norman Xiong (14679198) (LOCKED),Norman Xiong,9700,28.0
52,Chase Carlson (14679246) (LOCKED),Chase Carlson,6200,500.0
53,Steve Sear (14679247) (LOCKED),Steve Sear,6100,750.0
54,Li Wang (14679248) (LOCKED),Li Wang,6000,600.0


In [14]:
#Add projections
import pickle
pkl_filename = 'outlaw_proj_model.pkl'
outlaw_proj_model_pkl = open(pkl_filename, 'rb')
outlaw_proj_model = pickle.load(outlaw_proj_model_pkl)
outlaw_proj_model

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [15]:
from sklearn.preprocessing import PolynomialFeatures  
poly = PolynomialFeatures(degree = 4) 
df['Points'] = outlaw_proj_model.predict(poly.fit_transform(df[['Odds']]))
df.iloc[np.r_[0:4,-4:0]]

,Name + ID,Name,Salary,Odds,Points
0,John Catlin (14679194) (LOCKED),John Catlin,10700,8.0,7.383962
1,Matt Picanso (14679195) (LOCKED),Matt Picanso,10400,14.0,7.023074
2,Mark Anguiano (14679196) (LOCKED),Mark Anguiano,10200,16.0,6.904185
3,Alistair Docherty (14679197) (LOCKED),Alistair Docherty,9900,20.0,6.668501
51,Nick Cantlay (14679245) (LOCKED),Nick Cantlay,6200,400.0,-5.997651
52,Chase Carlson (14679246) (LOCKED),Chase Carlson,6200,500.0,-7.222029
53,Steve Sear (14679247) (LOCKED),Steve Sear,6100,750.0,-8.807209
54,Li Wang (14679248) (LOCKED),Li Wang,6000,600.0,-8.021154


In [16]:
#Add Win
df['Win'] = 1 / (1 + df.Odds)
df.head()

,Name + ID,Name,Salary,Odds,Points,Win
0,John Catlin (14679194) (LOCKED),John Catlin,10700,8.0,7.383962,0.111111
1,Matt Picanso (14679195) (LOCKED),Matt Picanso,10400,14.0,7.023074,0.066667
2,Mark Anguiano (14679196) (LOCKED),Mark Anguiano,10200,16.0,6.904185,0.058824
3,Alistair Docherty (14679197) (LOCKED),Alistair Docherty,9900,20.0,6.668501,0.047619
4,Norman Xiong (14679198) (LOCKED),Norman Xiong,9700,28.0,6.205432,0.034483


In [17]:
#Read in tee times
df_tees = pd.read_csv('Data/outlaw_tees.csv')
df_tees.head()

,Name,Tee,Hole
0,BA,NaN,NaN
1,Blake Abercrombie,2:00 PM,1.0
2,"Sacramento, CA",NaN,NaN
3,DA,NaN,NaN
4,Derek Ackerman,2:10 PM,1.0


In [18]:
df_tees = df_tees[df_tees.Tee.notnull()]
df_tees.iloc[np.r_[0:3,-3:0]]

,Name,Tee,Hole
1,Blake Abercrombie,2:00 PM,1.0
4,Derek Ackerman,2:10 PM,1.0
7,Blake Allen,12:40 PM,1.0
435,Willie Yacknow,7:30 AM,1.0
438,Phillip Yribarren,12:50 PM,10.0
441,Andrew Yun,1:20 PM,1.0


In [19]:
#replace names with what appears in df
for i in np.arange(len(df_tees)):
    curr_name = df_tees['Name'].iloc[i]
    try:
        new_name = get_close_matches(curr_name,df['Name'],cutoff=.85)[0]
        if new_name != curr_name:
            _
            print("{} replaced with {}".format(curr_name,new_name))
    except:
        new_name = 'No Name'
        #print('NO MATCH: curr_name = {}, new_name = {}'.format(curr_name,new_name))
    df_tees['Name'].iloc[i] = new_name
#df2['Name'] = df2['Name'].apply(lambda x: get_close_matches(x,df2_sal['Name']))
df_tees.head(5)

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,Name,Tee,Hole
1,Blake Abercrombie,2:00 PM,1.0
4,Derek Ackerman,2:10 PM,1.0
7,No Name,12:40 PM,1.0
10,No Name,2:10 PM,10.0
13,No Name,8:30 AM,1.0


In [20]:
#Get Delay from first tee time
t = pd.to_datetime(df_tees.Tee)
td_ser = t-t.min()
df_tees['tee_delay'] = [td.seconds/3600 for td in td_ser]

#Make Hole int
df_tees.Hole = df_tees.Hole.astype(int)


df_tees.head()

,Name,Tee,Hole,tee_delay
1,Blake Abercrombie,2:00 PM,1,6.500000
4,Derek Ackerman,2:10 PM,1,6.666667
7,No Name,12:40 PM,1,5.166667
10,No Name,2:10 PM,10,6.666667
13,No Name,8:30 AM,1,1.000000


In [21]:
#add this to df
df = pd.merge(df,df_tees,how='left')
df[df.Tee.isnull()]

,Name + ID,Name,Salary,Odds,Points,Win,Tee,Hole,tee_delay


In [22]:
#Add points adjusted, and get rid of nulls
#df['Points_adj'] = df.Points + 0.6*df.tee_delay
df['Points_adj']= df.Points - np.where(df.Tee.str.contains('AM'),2.77,0)

#Optional: add bonus for those starting on hole 1
df['Points_adj'] = np.where(df.Hole==10,df.Points_adj+1.78,df.Points_adj)

df = df.dropna()
df.iloc[10:40]

,Name + ID,Name,Salary,Odds,Points,Win,Tee,Hole,tee_delay,Points_adj
10,David Gazzolo (14679206) (LOCKED),David Gazzolo,8700,28.0,6.205432,0.034483,1:00 PM,1,5.500000,6.205432
11,Eric Sugimoto (14679207) (LOCKED),Eric Sugimoto,8600,33.0,5.921568,0.029412,8:40 AM,1,1.166667,3.151568
12,Chad Hambright (14679208) (LOCKED),Chad Hambright,8500,80.0,3.453690,0.012346,1:00 PM,10,5.500000,5.233690
13,Matt Marshall (14679209) (LOCKED),Matt Marshall,8400,50.0,4.987743,0.019608,1:20 PM,1,5.833333,4.987743
14,Noah Hofman (14679210) (LOCKED),Noah Hofman,8300,80.0,3.453690,0.012346,12:30 PM,1,5.000000,3.453690
15,Andrew Funk (14679211) (LOCKED),Andrew Funk,8200,40.0,5.531240,0.024390,1:50 PM,1,6.333333,5.531240
16,Zach Smith (14679212) (LOCKED),Zach Smith,8100,20.0,6.668501,0.047619,1:00 PM,1,5.500000,6.668501
17,Zander Winston (14679213) (LOCKED),Zander Winston,8000,40.0,5.531240,0.024390,9:20 AM,10,1.833333,4.541240
18,Mario Beltran (14679214) (LOCKED),Mario Beltran,7900,80.0,3.453690,0.012346,9:00 AM,1,1.500000,0.683690
19,Matt Snyder (14679215) (LOCKED),Matt Snyder,7800,80.0,3.453690,0.012346,8:00 AM,1,0.500000,0.683690


In [23]:
#Save resulting df
df.to_csv('Data/Outlaw_projections_20200625.csv',index=False)

In [24]:
df[df.Name.str.contains('Smith')]

,Name + ID,Name,Salary,Odds,Points,Win,Tee,Hole,tee_delay,Points_adj
16,Zach Smith (14679212) (LOCKED),Zach Smith,8100,20.0,6.668501,0.047619,1:00 PM,1,5.500000,6.668501
35,Scott Smith (14679234) (LOCKED),Scott Smith,6800,400.0,-5.997651,0.002494,12:50 PM,10,5.333333,-4.217651


In [25]:
df[df.Name.str.contains('Weaver')]

,Name + ID,Name,Salary,Odds,Points,Win,Tee,Hole,tee_delay,Points_adj
21,Michael Weaver (14679217) (LOCKED),Michael Weaver,7700,125.0,1.407923,0.007937,1:10 PM,10,5.666667,3.187923


# Explore yesterday

In [26]:
df1 = pd.read_csv(url1,usecols=['Name','Salary'])
df1.head(2)

,Name,Salary
0,John Catlin,10600
1,Rico Hoey,10300


In [27]:
#Read results
df_res = pd.read_csv('Data/outlaw_results_20200623.csv')
df_res.rename(columns={'Player':'Name'},inplace=True)
df_res.head(3)

,Rank,EntryId,EntryName,TimeRemaining,Points,Lineup,Unnamed: 6,Name,Roster Position,%Drafted,FPTS
0,1,1991621004,smittyuiop (30/38),0,392.10,G John Catlin G Eric Sugimoto G Derek Bayley G...,NaN,John Catlin,G,39.29%,57.60
1,2,1991816778,mts5200 (64/70),0,384.55,G John Catlin G Cody Blick G Andrew Yun G Matt...,NaN,Nicolo Galletti,G,35.54%,24.20
2,3,1992308369,Alext8,0,370.75,G Matt Picanso G Mark Anguiano G Cody Blick G ...,NaN,Sam Triplett,G,27.51%,26.55


In [28]:
#Add FPTS
df1 = pd.merge(df1,df_res[['Name','FPTS']],how='inner')
#Add Tees
df1 = pd.merge(df1,df_tees)
#Add odds
df1 = pd.merge(df1,df_odds)
df1.head(10)

,Name,Salary,FPTS,Tee,Hole,tee_delay,Odds
0,John Catlin,10600,57.60,8:40 AM,10,1.166667,8
1,Matt Picanso,10000,53.45,2:00 PM,10,6.500000,14
2,Mark Anguiano,9800,67.75,1:50 PM,1,6.333333,16
3,Brady Calkins,9600,53.25,9:30 AM,1,2.000000,16
4,Alistair Docherty,9200,46.90,1:10 PM,10,5.666667,20
5,Eric Sugimoto,8800,67.95,8:40 AM,1,1.166667,33
6,Norman Xiong,8700,64.95,1:10 PM,10,5.666667,28
7,David Gazzolo,8500,42.65,1:00 PM,1,5.500000,28
8,Andrew Funk,8400,32.20,1:50 PM,1,6.333333,40
9,Zander Winston,8200,44.95,9:20 AM,10,1.833333,40


In [29]:
#Add wave
df1['Wave'] = np.where(df1.Tee.str.contains('AM'),'AM','PM')

#Add points
df1['Points'] = outlaw_proj_model.predict(poly.fit_transform(df1[['Odds']]))
df1.head()



,Name,Salary,FPTS,Tee,Hole,tee_delay,Odds,Wave,Points
0,John Catlin,10600,57.60,8:40 AM,10,1.166667,8,AM,7.383962
1,Matt Picanso,10000,53.45,2:00 PM,10,6.500000,14,PM,7.023074
2,Mark Anguiano,9800,67.75,1:50 PM,1,6.333333,16,PM,6.904185
3,Brady Calkins,9600,53.25,9:30 AM,1,2.000000,16,AM,6.904185
4,Alistair Docherty,9200,46.90,1:10 PM,10,5.666667,20,PM,6.668501


In [30]:
g1 = df1.groupby('Wave')['Salary','Points','FPTS'].mean()
g1

,Salary,Points,FPTS
Wave,,,
AM,7458.333333,1.415226,58.260417
PM,7626.086957,2.456065,49.343478


In [31]:
#Find true wave split difference
g1.at['AM','FPTS'] - g1.at['PM','FPTS'] + (g1.at['PM','Points'] - g1.at['AM','Points'])

9.957778200009523

In [32]:
g2 = df1.groupby('Hole')['Salary','Points','FPTS'].mean()
g2

,Salary,Points,FPTS
Hole,,,
1,7428.571429,1.486510,54.741071
10,7705.263158,2.570138,52.652632


In [33]:
#Find true wave split difference
g2.at[1,'FPTS'] - g2.at[10,'FPTS'] + (g2.at[10,'Points'] - g2.at[1,'Points'])

3.172067880958628

In [34]:
df1.groupby('Wave')['Salary','FPTS'].mean().round(2)

,Salary,FPTS
Wave,,
AM,7458.33,58.26
PM,7626.09,49.34


In [35]:
df1.groupby('Hole')['Salary','FPTS'].mean().round(2)

,Salary,FPTS
Hole,,
1,7428.57,54.74
10,7705.26,52.65


In [36]:
df1.groupby(['Wave','Hole'])['Salary','FPTS'].mean().round(2)

Salary   FPTS
Wave Hole                
AM   1     7358.33  61.80
     10    7558.33  54.73
PM   1     7481.25  49.45
     10    7957.14  49.10

In [37]:
from IPython.core.display import display, HTML

In [38]:
html = "Outlaw Round 1 by Wave and Starting Hole"
html += df1.groupby('Wave')['Salary','FPTS'].mean().round(2).to_html()
html += df1.groupby('Hole')['Salary','FPTS'].mean().round(2).to_html()
html += df1.groupby(['Wave','Hole'])['Salary','FPTS'].mean().round(2).to_html()
display(HTML(html))

# PGA showdown

In [1]:
import pandas as pd
import numpy as np

In [2]:
url = 'Data/draftkings_showdown_projections.csv'
df= pd.read_csv(url)
df.tail()

,datagolf_name,dk_name,dk_id,sample_size,dk_salary,r4_tee_time,morning_wave,position,scoring_points,finish_points,manual_bump,total_points,value,max_exposure,lineup_rule
65,"Baker, Chris",Chris Baker,14722512,150,6200,12:15 PM,0,T19 (-11),39.4,2.8,0,42.2,0.27,100,NaN
66,"Gellerman, Michael",Michael Gellerman,14722514,150,6100,11:35 AM,0,T30 (-10),36.3,1.7,0,38.0,-0.27,100,NaN
67,"Byrd, Jonathan",Jonathan Byrd,14722513,150,6100,11:35 AM,0,T30 (-10),36.4,1.7,0,38.1,-0.25,100,NaN
68,"McNeill, George",George McNeill,14722515,150,6000,9:05 AM,1,T59 (-7),36.0,0.4,0,36.4,-0.48,100,NaN
69,"Van Pelt, Bo",Bo Van Pelt,14722516,150,6000,8:35 AM,1,T68 (-4),33.2,0.0,0,33.3,-1.00,100,NaN


In [3]:
teecol = [c for c in df.columns if c.endswith('tee_time')][0]
df.rename(columns={teecol:'tee'},inplace=True)
df.tee = df.tee.str.replace('*','')
df.tee = pd.to_datetime(df.tee)

td_ser = df.tee-df.tee.min()
df['delay'] = [td.seconds/3600 for td in td_ser]
df.head()

,datagolf_name,dk_name,dk_id,sample_size,dk_salary,tee,morning_wave,position,scoring_points,finish_points,manual_bump,total_points,value,max_exposure,lineup_rule,delay
0,"DeChambeau, Bryson",Bryson DeChambeau,14722447,150,10800,2020-07-05 13:55:00,0,T2 (-16),48.9,9.4,0,58.3,0.72,100,NaN,5.500000
1,"Simpson, Webb",Webb Simpson,14722448,150,10500,2020-07-05 13:15:00,0,T8 (-13),47.9,6.4,0,54.3,0.35,100,NaN,4.833333
2,"Hovland, Viktor",Viktor Hovland,14722449,150,10200,2020-07-05 13:25:00,0,T8 (-13),46.5,6.1,0,52.6,0.23,100,NaN,5.000000
3,"Hatton, Tyrrell",Tyrrell Hatton,14722450,150,10000,2020-07-05 12:45:00,0,T13 (-12),46.9,5.2,0,52.1,0.25,100,NaN,4.333333
4,"Matsuyama, Hideki",Hideki Matsuyama,14722451,150,9800,2020-07-05 13:05:00,0,T13 (-12),48.1,5.5,0,53.6,0.48,100,NaN,4.666667


In [4]:
#add points adjusted
df['points_adj']=df.total_points -0.6*df.delay
df.head()

,datagolf_name,dk_name,dk_id,sample_size,dk_salary,tee,morning_wave,position,scoring_points,finish_points,manual_bump,total_points,value,max_exposure,lineup_rule,delay,points_adj
0,"DeChambeau, Bryson",Bryson DeChambeau,14722447,150,10800,2020-07-05 13:55:00,0,T2 (-16),48.9,9.4,0,58.3,0.72,100,NaN,5.500000,55.0
1,"Simpson, Webb",Webb Simpson,14722448,150,10500,2020-07-05 13:15:00,0,T8 (-13),47.9,6.4,0,54.3,0.35,100,NaN,4.833333,51.4
2,"Hovland, Viktor",Viktor Hovland,14722449,150,10200,2020-07-05 13:25:00,0,T8 (-13),46.5,6.1,0,52.6,0.23,100,NaN,5.000000,49.6
3,"Hatton, Tyrrell",Tyrrell Hatton,14722450,150,10000,2020-07-05 12:45:00,0,T13 (-12),46.9,5.2,0,52.1,0.25,100,NaN,4.333333,49.5
4,"Matsuyama, Hideki",Hideki Matsuyama,14722451,150,9800,2020-07-05 13:05:00,0,T13 (-12),48.1,5.5,0,53.6,0.48,100,NaN,4.666667,50.8


In [5]:
df.to_csv('Data/dkshowdown.csv')